In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

/Library/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
##load the data , it is huge. Do not read again 
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
client_names = pd.read_csv("cliente_tabla.csv")
product_names = pd.read_csv("producto_tabla.csv")
town = pd.read_csv("town_state.csv")

In [3]:
columnsDict = {'Semana': 'week_num', 'Agencia_ID': 'sales_depot_ID', 'Canal_ID':'sales_channel_ID',
               'Ruta_SAK': 'route_ID', 'Cliente_ID': 'client_ID', 'NombreCliente': 'client_name', 
               'Producto_ID': 'product_ID', 'Venta_uni_hoy': 'sales_unit_this_week', 
               'Venta_hoy': 'sales_this_week', 'Dev_uni_proxima': 'returns_unit_next_week', 
               'Dev_proxima': 'returns_next_week', 'Demanda_uni_equil': 'adjusted_demand'}
train.rename(columns = columnsDict, inplace = True)
test.rename(columns = columnsDict, inplace = True)
client_names.rename(columns = columnsDict, inplace = True)
town.rename(columns = columnsDict, inplace = True)
train.head()

,week_num,sales_depot_ID,sales_channel_ID,route_ID,client_ID,product_ID,sales_unit_this_week,sales_this_week,returns_unit_next_week,returns_next_week,adjusted_demand
0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3


In [20]:
##The product in train data set is not fully present in test data. Check how many are they 
def differenceIDTwoDF(train, test):
    test_prod_id = set(test.product_ID.unique())
    train_prod_id = set(train.product_ID.unique())
    intersection  = (test_prod_id & train_prod_id)
    print('train data product id number:', len(train_prod_id))
    print('test data product id number:', len(test_prod_id))
    print('intersection number:', len(intersection))
    unintersection = test_prod_id - intersection
    print('difference:', len(unintersection), unintersection)

In [21]:
differenceIDTwoDF(train, test)

('train data product id number:', 1799)
('test data product id number:', 1522)
('intersection number:', 1488)
('difference:', 34, set([32026, 37404, 37405, 32798, 32421, 37745, 36524, 37618, 35246, 33053, 46131, 32820, 37688, 36673, 37702, 35191, 32591, 37202, 42323, 48217, 32224, 98, 31203, 37610, 31211, 46064, 37617, 37362, 37620, 31655, 37494, 37495, 37496, 37626]))


In [38]:
## train data week num is from 3-9, and test week num is 10 and 11
test_data = train[train.week_num.isin([9])]
train_data = train[~train.week_num.isin([9])]
print("training set number: ", train_data.shape[0])
print("test set number: ", test_data.shape[0])

('training set number: ', 63771751)
('test set number: ', 10408713)


In [22]:
differenceIDTwoDF(train_data, test_data)

('train data product id number:', 1758)
('test data product id number:', 1566)
('intersection number:', 1525)
('difference:', 41, set([35713, 36995, 43398, 37639, 36873, 48138, 37264, 37139, 48023, 37402, 40987, 37149, 36131, 35240, 48297, 35761, 3509, 33208, 37689, 37690, 36671, 37403, 33735, 37576, 4169, 42192, 37587, 37590, 36699, 37468, 37469, 37470, 40929, 35304, 35306, 35310, 35441, 35442, 37363, 37242, 1277]))


In [28]:
##only pick 50000 users 
import random

def pickRandomData(data, numPick, pool):
    unique_prod_id = pool
    print("Number of unique product id ",unique_prod_id.shape[0])
    sel_ids = [unique_prod_id[i] for i in sorted(random.sample(range(len(unique_prod_id)), numPick)) ]
    sel_data = data[data.product_ID.isin(sel_ids)]
    return sel_data

In [39]:
##random sample the data 
import random

def pickRandomData(data, numPick):
    sel_data = data.sample(numPick)
    return sel_data

In [48]:
small_train = pickRandomData(train_data, 6377175)
small_test = pickRandomData(test_data, 1040871)

In [49]:
differenceIDTwoDF(small_train, small_test)

('train data product id number:', 1526)
('test data product id number:', 1292)
('intersection number:', 1246)
('difference:', 46, set([37379, 31621, 48138, 37139, 37402, 37403, 37149, 37153, 35110, 48039, 31669, 33208, 37689, 37690, 32223, 36671, 49989, 37576, 49993, 37578, 4301, 47066, 37590, 31716, 49754, 37469, 37470, 34783, 40929, 36995, 33381, 35304, 4220, 35306, 107, 35310, 1277, 35442, 37363, 31861, 36058, 37241, 37242, 4859, 46076, 43389]))


In [50]:
import cPickle as pickle
pickle_file = 'dataset.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'small_test': small_test,
    'small_train': small_train,
    'client_names': client_names, 
    'town': town
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [1]:
from six.moves import cPickle as pickle

pickle_file = 'dataset.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  small_test = save['small_test']
  small_train = save['small_train']
  client_names = save['client_names']
  town = save['town']
  del save  # hint to help gc free up memory
  print('Small training set', small_train.shape)
  print('Small test set', small_test.shape)
  print('destinations', client_names.shape)
  print('town', town.shape)

('Small training set', (6377175, 11))
('Small test set', (1040871, 11))
('destinations', (935362, 2))
('town', (790, 3))


In [ ]:
###could use xgboost with the customize objective funciton
## evalerror <- function(preds, dtrain) {
#   labels <- getinfo(dtrain, "label")
#   #preds[preds<0] <- 0
#   preds <- abs(preds)
#   err <- rmsell(preds, labels)
#   return(list(metric = "rmsell", value = err))
# }


# def evalerror(preds, dtrain):
#     labels = dtrain.get_label()
#     assert len(preds) == len(labels)
#     labels = labels.tolist()
#     preds = preds.tolist()
#     # I have added the max since applying regression we obtain negative values of preds
#     # and therefore an error because of the logarithm
#     terms_to_sum = [(math.log(labels[i] + 1) - math.log(max(0,preds[i]) + 1)) ** 2.0 
#                     for i,pred in enumerate(labels)]
#     return 'error', (sum(terms_to_sum) * (1.0/len(preds))) ** 0.5

# xgclassifier = xgb.train(params, xg_train, num_round, watchlist, 
#                      early_stopping_rounds=early_stopping, 
#                      feval = evalerror, verbose_eval = 1);
##https://www.kaggle.com/vykhand/grupo-bimbo-inventory-demand/exploring-products/discussion 
## https://gist.github.com/rowanv/f555f65721f5bf730053
## https://www.kaggle.com/kobakhit/grupo-bimbo-inventory-demand/xgboost/comments

## https://www.kaggle.com/kelexu/grupo-bimbo-inventory-demand/0-48881/code